<a href="https://colab.research.google.com/github/timotheekurniawan/emotion_recognition/blob/master/vgg_with_augmentation_undersampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

USING VGG MODEL ,  WITH UNDERSAMPLING AND DATA AUGMENTATION

In [13]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
import io
from keras.preprocessing.image import ImageDataGenerator


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
dataset=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/fer2013.csv")


In [4]:
dataset = dataset.loc[dataset["emotion"]!=1]
def balancingDataset(x):
  new_df=dataset.loc[dataset['emotion']==x]
  new_df=new_df[:4000]
  return new_df
df_0=balancingDataset(0)
df_2=balancingDataset(2)
df_3=balancingDataset(3)
df_4=balancingDataset(4)
df_5=balancingDataset(5)
df_6=balancingDataset(6)


dataset=pd.concat([df_0,df_2,df_3,df_4,df_5,df_6])
dataset

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
10,0,30 24 21 23 25 25 49 67 84 103 120 125 130 139...,Training
22,0,123 125 124 142 209 226 234 236 231 232 235 22...,Training
23,0,8 9 14 21 26 32 37 46 52 62 72 70 71 73 76 83 ...,Training
...,...,...,...
23047,6,252 252 252 251 252 249 247 242 208 211 142 81...,Training
23051,6,26 21 56 134 148 166 177 188 192 185 186 181 1...,Training
23056,6,42 53 51 47 34 29 27 42 50 41 42 40 44 35 58 9...,Training
23057,6,143 141 133 131 128 126 125 120 121 115 55 54 ...,Training


In [5]:
def convert_pixels_to_img(pixels):
    test_image = pixels.reshape(48,48,1)
    return test_image

def split_pixels(string):
    splitted = np.array(string.split(),'int')
    return splitted 

def change_to_categorical(sample):
    return keras.utils.to_categorical(sample,num_classes=7)


In [6]:
dataset['pixels'] = dataset['pixels'].apply(split_pixels)
dataset['label'] = dataset['emotion'].apply(change_to_categorical)
dataset['length'] = dataset['pixels'].apply(len)

dataset = dataset[dataset.length == 2304]
dataset['image'] = dataset['pixels'].apply(convert_pixels_to_img)

In [7]:
grouped_dataset = dataset.groupby(dataset.Usage)
training_dataset = grouped_dataset.get_group("Training")
dev_dataset = grouped_dataset.get_group("PublicTest")
test_dataset = grouped_dataset.get_group("PrivateTest")

In [8]:
model = keras.Sequential()
model.add(layers.Conv2D(input_shape=(48,48,1),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(units=4096,activation="relu"))
model.add(layers.Dense(units=4096,activation="relu"))
model.add(layers.Dense(units=7, activation="softmax"))
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 256)       2

In [9]:
x_train = training_dataset['image']
y_train = training_dataset['label']
x_test =  dev_dataset['image']
y_test = dev_dataset['label']

x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()


x_train = np.stack(x_train,axis=0)
y_train = np.stack(y_train,axis=0)
x_test = np.stack(x_test,axis=0)
y_test = np.stack(y_test,axis=0)

In [14]:
datagen = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        shear_range = 10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)


In [16]:
model.fit(datagen.flow(x_train, y_train, batch_size=32),batch_size=32,epochs=150,verbose=1,validation_data=(x_test, y_test))


Epoch 1/150
724/724 [==============================] - 19s 26ms/step - loss: 0.7817 - accuracy: 0.7065 - val_loss: 0.5811 - val_accuracy: 0.8188
Epoch 2/150
724/724 [==============================] - 18s 25ms/step - loss: 0.7741 - accuracy: 0.7081 - val_loss: 0.4321 - val_accuracy: 0.8768
Epoch 3/150
724/724 [==============================] - 18s 25ms/step - loss: 0.7617 - accuracy: 0.7131 - val_loss: 0.7329 - val_accuracy: 0.7633
Epoch 4/150
724/724 [==============================] - 18s 25ms/step - loss: 0.7547 - accuracy: 0.7168 - val_loss: 0.7139 - val_accuracy: 0.7874
Epoch 5/150
724/724 [==============================] - 18s 25ms/step - loss: 0.7442 - accuracy: 0.7207 - val_loss: 0.6368 - val_accuracy: 0.8213
Epoch 6/150
724/724 [==============================] - 18s 25ms/step - loss: 0.7321 - accuracy: 0.7224 - val_loss: 0.5669 - val_accuracy: 0.8285
Epoch 7/150
724/724 [==============================] - 18s 25ms/step - loss: 0.7252 - accuracy: 0.7284 - val_loss: 0.5021 - val_ac

In [21]:
# x_test =  test_dataset['image']
# y_test = test_dataset['label']

# x_test = x_test.to_numpy()
# y_test = y_test.to_numpy()


# x_test = np.stack(x_test,axis=0)
# y_test = np.stack(y_test,axis=0)

testloss = model.evaluate(x_test, y_test) 
print("Test Loss " + str(testloss[0]))
print("Test Acc: " + str(testloss[1]))
trainloss = model.evaluate(x_train, y_train) 
print("Train Loss " + str(trainloss[0]))
print("Train Acc: " + str(trainloss[1]))

13/13 [==============================] - 0s 9ms/step - loss: 1.1826 - accuracy: 0.8575
Test Loss 1.1825506687164307
Test Acc: 0.8574879169464111
724/724 [==============================] - 6s 9ms/step - loss: 0.0745 - accuracy: 0.9731
Train Loss 0.07451494038105011
Train Acc: 0.9731071591377258
